In [1]:
import os

from keras import Input, Model
from keras import optimizers, regularizers
from keras.callbacks import ModelCheckpoint
from keras.datasets import cifar10
from keras.layers import (add, Conv2D, GlobalAveragePooling2D)
from sklearn.model_selection import train_test_split
from tensorflow import keras

os.environ['KERAS_BACKEND'] = 'tensorflow'

import numpy as np
from keras.datasets import cifar100
from keras.layers import Dense, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.optimizers import SGD

In [2]:
import freezing
import resnet_sd

In [4]:
# Per channel mean and std normalization
def color_preprocessing(x_train, x_val, x_test):
    
    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')    
    x_test = x_test.astype('float32')
    
    mean = np.mean(x_train, axis=(0,1,2))  # Per channel mean
    std = np.std(x_train, axis=(0,1,2))
    x_train = (x_train - mean) / std
    x_val = (x_val - mean) / std
    x_test = (x_test - mean) / std
    
    return x_train, x_val, x_test

In [ ]:
img_rows, img_cols = 32, 32
img_channels = 3
nb_epochs = 500
batch_size = 128
nb_classes = 100
seed = 333


# data
(x_train, y_train), (x_test, y_test) = cifar100.load_data()


# Data splitting (get additional 5k validation set)
# Sklearn to split
x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=seed)  # random_state = seed
x_train45, x_val, x_test = color_preprocessing(x_train45, x_val, x_test)


img_gen = ImageDataGenerator(
    horizontal_flip=True,
    data_format="channels_last",
    width_shift_range=0.125,  # 0.125*32 = 4 so max padding of 4 pixels, as described in paper.
    height_shift_range=0.125,
    fill_mode="constant",
    cval = 0
)

img_gen.fit(x_train45)

y_train45 = np_utils.to_categorical(y_train45, nb_classes)  # 1-hot vector
y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)


# building and training net
model = resnet_sd.resnet_sd_model(img_shape = (32,32), img_channels = 3, layers = 110, nb_classes = nb_classes, verbose = True)

sgd = SGD(learning_rate=0.1, decay=1e-4, momentum=0.9, nesterov=True)

checkpointer = ModelCheckpoint('model_resnet110SD_c100_best_2.hdf5', verbose=1, save_best_only=True)

In [6]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.9:
    freezing_list.append(int(nb_epochs*0.6))
  elif i < len(model.layers) * 0.98:
    freezing_list.append(int(nb_epochs*0.96))
freezing_list.append(nb_epochs)

In [7]:
freezing.training_with_freezing(model, img_gen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list, batch_size=128,lr_schedule = [[0, 0.1],[nb_epochs*0.5,0.01],[nb_epochs*0.75,0.001]],cbks=[checkpointer], name='resnet_sd_cifar100_2')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
......activation_25
.........vars
......activation_26
.........vars
......activation_27
.........vars
......activation_28
.........vars
......activation_29
.........vars
......activation_3
.........vars
......activation_30
.........vars
......activation_31
.........vars
......activation_32
.........vars
......activa

[25.539999999999992,
 0.14255762116760012,
 0.3067331643514736,
 1.2519651429346974,
 0.22999444347230427]

In [ ]:
stack_n = 18
num_classes = 10
img_rows, img_cols = 32, 32
img_channels = 3
batch_size = 128
epochs = 200 
iterations = 45000 // batch_size
weight_decay = 0.0001
seed = 333


def residual_network(img_input, classes_num=10, stack_n=5):
    def residual_block(intput, out_channel, increase=False):
        if increase:
            stride = (2, 2)
        else:
            stride = (1, 1)

        pre_bn = BatchNormalization()(intput)
        pre_relu = Activation('relu')(pre_bn)

        conv_1 = Conv2D(out_channel, kernel_size=(3, 3), strides=stride, padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(pre_relu)
        bn_1 = BatchNormalization()(conv_1)
        relu1 = Activation('relu')(bn_1)
        conv_2 = Conv2D(out_channel, kernel_size=(3, 3), strides=(1, 1), padding='same',
                        kernel_initializer="he_normal",
                        kernel_regularizer=regularizers.l2(weight_decay))(relu1)
        if increase:
            projection = Conv2D(out_channel,
                                kernel_size=(1, 1),
                                strides=(2, 2),
                                padding='same',
                                kernel_initializer="he_normal",
                                kernel_regularizer=regularizers.l2(weight_decay))(intput)
            block = add([conv_2, projection])
        else:
            block = add([intput, conv_2])
        return block

    # build model
    # total layers = stack_n * 3 * 2 + 2
    # stack_n = 5 by default, total layers = 32
    # input: 32x32x3 output: 32x32x16
    x = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), padding='same',
               kernel_initializer="he_normal",
               kernel_regularizer=regularizers.l2(weight_decay))(img_input)

    # input: 32x32x16 output: 32x32x16
    for _ in range(stack_n):
        x = residual_block(x, 16, False)

    # input: 32x32x16 output: 16x16x32
    x = residual_block(x, 32, True)
    for _ in range(1, stack_n):
        x = residual_block(x, 32, False)

    # input: 16x16x32 output: 8x8x64
    x = residual_block(x, 64, True)
    for _ in range(1, stack_n):
        x = residual_block(x, 64, False)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)

    # input: 64 output: 10
    x = Dense(classes_num, activation='softmax',
              kernel_initializer="he_normal",
              kernel_regularizer=regularizers.l2(weight_decay))(x)
    return x



(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train45, x_val, y_train45, y_val = train_test_split(x_train, y_train, test_size=0.1,
                                                      random_state=seed)  # random_state = seed

img_mean = x_train45.mean(axis=0)  # per-pixel mean, what about std?
img_std = x_train45.std(axis=0)
x_train45 = (x_train45 - img_mean) / img_std
x_val = (x_val - img_mean) / img_std
x_test = (x_test - img_mean) / img_std

# build network
img_input = Input(shape=(img_rows, img_cols, img_channels))
output = residual_network(img_input, num_classes, stack_n)
model = Model(img_input, output)
print(model.summary())

# set optimizer
sgd = optimizers.SGD(learning_rate=.1, momentum=0.9, nesterov=True)
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant', cval=0.)
datagen.fit(x_train45)

In [9]:
freezing_list = []
for i in range(len(model.layers)):
  if i < len(model.layers) * 0.9:
    freezing_list.append(int(epochs*0.6))
  elif i < len(model.layers) * 0.98:
    freezing_list.append(int(epochs*0.96))
freezing_list.append(epochs)
checkpointer = ModelCheckpoint('model_resnet_c10_best_2.hdf5', verbose=1, save_best_only=True)

In [10]:
freezing.training_with_freezing(model, datagen, sgd, x_train45, y_train45, x_val, y_val, x_test, y_test,freezing_list, batch_size=128,lr_schedule = [[0, 0.1],[80,0.01],[150,0.001]],cbks=[checkpointer], name='resnet_cifar10_2')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......activation
.........vars
......activation_1
.........vars
......activation_10
.........vars
......activation_100
.........vars
......activation_101
.........vars
......activation_102
.........vars
......activation_103
.........vars
......activation_104
.........vars
......activation_105
.........vars
......activation_106
.........vars
......activation_107
.........vars
......activation_108
.........vars
......activation_11
.........vars
......activation_12
.........vars
......activation_13
.........vars
......activation_14
.........vars
......activation_15
.........vars
......activation_16
.........vars
......activation_17
.........vars
......activation_18
.........vars
......activation_19
.........vars
......activation_2
.........vars
......activation_20
.........vars
......activation_21
.........vars
......activation_22
.........vars
......activation_23
.........vars
......activation_24
.........vars
..

[6.930000000000007,
 0.04833229455351825,
 0.3222891390323639,
 0.32020646726851515,
 0.06022057364990712]